# Area Segmentation for CoastSat/GEE Compatibility   

- Create a polygon boundary of the **RoI (Region of Interest)**
    > *That can be easily achived through the web app [geojson.io](https://geojson.io/).*
- Save the large polygon boundary as the **input.json** file.**
    > *The input can be either a `Polygon` geometry or a `LineString` that forms a closed loop (ring). The script automatically treats a closed LineString as a polygon boundary.*

- **Automatic split/subdivision** of the input polygon into parts, based on:
    > - `WGS84 ellipsoid`: *The global standard used by [geojson.io](https://geojson.io/).*
    > - `100 sq km threshold`: *GEE requests may fail if the polygon is larger than 100 sq km.*
    > - `Non-overlapping` *resulting polygons.*

- **Automatic export** of each resulting segment:
    > - *Into a `/segments` directory.*
    > - *With properites of*:
    >     - *Ascendign `FID` identifier.*
    >     - *Area in `sq km`.*

- **Automatic combine** of all segments into a single json for:
    > - *`Visualization`*
    > - *`Confirmation`*

---

## ROI GeoJson Visualization using Python

In [1]:
!pip install folium geopandas

  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.0 kB)
  Using cached charset_normalizer-3.4.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 8.7 MB/s eta 0:00:00:00:01
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 20.0 MB/s eta 0:00:0000:0100:01
Using cached requ

In [2]:
import folium
import json
import geopandas as gpd

# Load GeoJSON as GeoDataFrame for easy bounds calculation
gdf = gpd.read_file('input.json')

# Calculate bounds: (minx, miny, maxx, maxy)
bounds = gdf.total_bounds
minx, miny, maxx, maxy = bounds

# Create map object
m = folium.Map()

# Add GeoJSON overlay
folium.GeoJson(
    gdf,
    name='geojson',
    style_function=lambda feature: {
        'fillColor': 'blue',
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.5,
    }
).add_to(m)


# Automatically fit map to GeoJSON bounds
m.fit_bounds([[miny, minx], [maxy, maxx]])

# Add layer control
folium.LayerControl().add_to(m)

# Save to HTML
m.save('interactive_map.html')

# Display the map
display(m)

## Area Calculation

Calculate the geodetic area of a given `input.json`, functioning as a polygon boundary.



In [ ]:
# Install necessary libraries
!pip install pyproj

In [29]:
import json
import pyproj

# Load GeoJSON from a file
with open('input.json', 'r') as f:
    geojson_file = json.load(f)

# If FeatureCollection, extract first feature
if geojson_file.get('type') == 'FeatureCollection':
    feature = geojson_file['features'][0]
elif geojson_file.get('type') == 'Feature':
    feature = geojson_file
else:
    raise ValueError("Unsupported GeoJSON format")

geometry = feature['geometry']
geom_type = geometry['type']
coords = geometry['coordinates']

# If LineString, convert to closed polygon ring
if geom_type == 'LineString':
    if coords[0] != coords[-1]:
        coords.append(coords[0])
    coords = [coords]  # wrap in list to mimic Polygon exterior ring
elif geom_type == 'Polygon':
    coords = coords
else:
    raise ValueError("Geometry type must be Polygon or LineString")

# Use the outer ring for area calculation
coordinates = coords[0]
geometry['coordinates'] = coords

# --- Area Calculation ---

# 1. Extract the coordinates
# GeoJSON format: [longitude, latitude]
# The coordinates list is nested: geometry -> coordinates -> outer_ring -> points
try:
    # Use the coordinates we already extracted
    # 1. Separate longitude and latitude into lists
    lons = [point[0] for point in coordinates]
    lats = [point[1] for point in coordinates]

    # 2. Define the geodetic reference system (WGS84 ellipsoid)
    # pyproj.Geod handles calculations on the ellipsoid
    geod = pyproj.Geod(ellps='WGS84')

    # 3. Calculate the area
    # polygon_area_perimeter returns area (m^2) and perimeter (m)
    # We only need the area. Use abs() as the sign depends on vertex order.
    area_m2, perimeter_m = geod.polygon_area_perimeter(lons, lats)
    area_m2 = abs(area_m2) # Ensure area is positive

    # 4. Convert area from square meters to square kilometers
    area_km2 = area_m2 / 1_000_000 # 1 km = 1000 m, so 1 km^2 = 1,000,000 m^2

    # 5. Print the result
    print(f"Calculated Area:")  # Green title
    print(f"  {area_m2:.2f} m²")
    print(f"  {area_km2:.2f} km²")
except KeyError as e:
    print(f"Error: Could not find key {e} in the GeoJSON data. Is it valid?")
except IndexError:
    print("Error: Could not extract coordinates. Check the GeoJSON structure.")
except ImportError:
    print("Error: The 'pyproj' library is required. Please install it (`pip install pyproj`).")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Calculated Area:
  1438448985.34 m²
  1438.45 km²


## Segmentation of the ROI (Region of Interest)

In [3]:
!pip install shapely

In [4]:
import json
import os
from shapely.geometry import Polygon, LineString, box, mapping
from shapely.ops import split
from shapely import affinity
from pyproj import Geod

INPUT_FILE = "input.json"  # Input polygon GeoJSON file
OUTPUT_DIR = "segments"    # Output directory for polygon segments
TARGET_AREA_KM2 = 100
geod = Geod(ellps="WGS84")

def calculate_area_km2(poly):
    lon, lat = poly.exterior.coords.xy
    area, _ = geod.polygon_area_perimeter(lon, lat)
    return abs(area) / 1_000_000

def recursive_split(polygon, fid_counter):
    """
    Recursively split polygon into parts < TARGET_AREA_KM2.
    fid_counter: list with one int element, incremented for each saved segment.
    Returns list of (fid, polygon, area).
    """
    queue = [polygon]
    segments = []

    while queue:
        poly = queue.pop(0)
        area = calculate_area_km2(poly)
        if area <= TARGET_AREA_KM2:
            fid = fid_counter[0]
            fid_counter[0] += 1
            segments.append((fid, poly, area))
        else:
            # Split polygon vertically at center
            minx, miny, maxx, maxy = poly.bounds
            midx = (minx + maxx) / 2
            splitter = LineString([(midx, miny - 1), (midx, maxy + 1)])
            parts = split(poly, splitter)
            # Normalize parts to list of polygons
            if parts.geom_type == 'GeometryCollection':
                parts = [g for g in parts.geoms if g.geom_type == 'Polygon']
            elif parts.geom_type == 'Polygon':
                parts = [parts]
            elif parts.geom_type == 'MultiPolygon':
                parts = list(parts.geoms)
            else:
                parts = []

            # If split failed (1 part), try horizontal split
            if len(parts) <= 1:
                midy = (miny + maxy) / 2
                splitter = LineString([(minx - 1, midy), (maxx + 1, midy)])
                parts = split(poly, splitter)
                if parts.geom_type == 'GeometryCollection':
                    parts = [g for g in parts.geoms if g.geom_type == 'Polygon']
                elif parts.geom_type == 'Polygon':
                    parts = [parts]
                elif parts.geom_type == 'MultiPolygon':
                    parts = list(parts.geoms)
                else:
                    parts = []

            # If still 1 part, stop splitting
            if len(parts) <= 1:
                fid = fid_counter[0]
                fid_counter[0] += 1
                segments.append((fid, poly, area))
            else:
                queue.extend(parts)
    return segments

def main():
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    with open(INPUT_FILE) as f:
        data = json.load(f)

    coords = data['features'][0]['geometry']['coordinates']
    # Close polygon if not closed
    if coords[0] != coords[-1]:
        coords.append(coords[0])

    polygon = Polygon(coords)

    fid_counter = [0]
    segments = recursive_split(polygon, fid_counter)

    for fid, poly, area in segments:
        feature = {
            "type": "Feature",
            "properties": {"FID": fid, "area_km2": area},
            "geometry": mapping(poly)
        }
        out_path = os.path.join(OUTPUT_DIR, f"segment_{fid}.json")
        with open(out_path, "w") as f_out:
            json.dump({
                "type": "FeatureCollection",
                "features": [feature]
            }, f_out, indent=2)
        print(f"Saved segment {fid} with area {area:.2f} km²")

    print(f"Total segments created: {len(segments)}")

if __name__ == "__main__":
    main()

Saved segment 0 with area 89.51 km²
Saved segment 1 with area 14.07 km²
Saved segment 2 with area 9.20 km²
Saved segment 3 with area 83.59 km²
Saved segment 4 with area 75.99 km²
Saved segment 5 with area 6.06 km²
Saved segment 6 with area 63.60 km²
Saved segment 7 with area 64.39 km²
Saved segment 8 with area 61.88 km²
Saved segment 9 with area 86.86 km²
Saved segment 10 with area 29.80 km²
Saved segment 11 with area 77.63 km²
Saved segment 12 with area 66.08 km²
Saved segment 13 with area 39.74 km²
Saved segment 14 with area 62.49 km²
Saved segment 15 with area 45.02 km²
Saved segment 16 with area 17.97 km²
Saved segment 17 with area 41.11 km²
Saved segment 18 with area 51.16 km²
Saved segment 19 with area 24.72 km²
Saved segment 20 with area 67.38 km²
Saved segment 21 with area 63.45 km²
Saved segment 22 with area 7.02 km²
Saved segment 23 with area 31.27 km²
Saved segment 24 with area 35.31 km²
Saved segment 25 with area 68.88 km²
Saved segment 26 with area 25.12 km²
Saved segment 

## Combination of Segments for Confirmation

In [5]:
import json
import glob
import os

SEGMENTS_DIR = "segments"  # Directory containing segment GeoJSON files
OUTPUT_FILE = "combined_segments.json"  # Output combined GeoJSON file

def main():
    features = []

    # Find all segment files
    segment_files = sorted(glob.glob(os.path.join(SEGMENTS_DIR, "segment_*.json")))

    for seg_file in segment_files:
        with open(seg_file) as f:
            data = json.load(f)
            for feature in data.get("features", []):
                features.append(feature)

    # Sort features by FID
    features.sort(key=lambda f: f.get("properties", {}).get("FID", 0))

    combined = {
        "type": "FeatureCollection",
        "features": features
    }

    with open(OUTPUT_FILE, "w") as f:
        json.dump(combined, f, indent=2)

    print(f"Combined {len(features)} segments into '{OUTPUT_FILE}'")

if __name__ == "__main__":
    main()

Combined 31 segments into 'combined_segments.json'


## Visualize the Combined Segments

In [6]:
import folium
import json
import geopandas as gpd

# Load GeoJSON as GeoDataFrame for easy bounds calculation
gdf = gpd.read_file('combined_segments.json')

# Calculate bounds: (minx, miny, maxx, maxy)
bounds = gdf.total_bounds
minx, miny, maxx, maxy = bounds

# Create map object
m = folium.Map()

# Add GeoJSON overlay
folium.GeoJson(
    gdf,
    name='geojson',
    style_function=lambda feature: {
        'fillColor': 'blue',
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.5,
    }
).add_to(m)


# Automatically fit map to GeoJSON bounds
m.fit_bounds([[miny, minx], [maxy, maxx]])

# Add layer control
folium.LayerControl().add_to(m)

# Save to HTML
m.save('interactive_map.html')

# Display the map
display(m)